<a href="https://colab.research.google.com/github/PhDenLogica/DiaDeLiberacionEstatal/blob/main/DiaDeLiberacionEstatal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DIA DE LIBERACION ESTATAL



In [ ]:
#@title Apreta el boton de play que esta aca abajo a la izquierda para empezar
import pandas as pd
from datetime import datetime, timedelta

def obtener_float(mensaje, default=0):
    print(mensaje + f" (Default: {default}):")  # Print the message
    entrada = input() or default  # Input on a new line
    return float(entrada)

def obtener_bool(mensaje):
    print(mensaje + " (s/n):")  # Print the message
    entrada = input().strip().lower()  # Input on a new line
    return entrada == 's'

def calcular_irpf(base_irpf):
    # Definir las franjas del IRPF
    franjas = [
        (0, 46032, 0.00),
        (46033, 65760, 0.10),
        (65761, 98640, 0.15),
        (98641, 197280, 0.24),
        (197281, 328800, 0.25),
        (328801, 493200, 0.27),
        (493201, 756240, 0.31),
        (756241, float('inf'), 0.36)
    ]

    irpf = 0  # Inicializar el impuesto total

    # Calcular el impuesto por franjas
    for i, franja in enumerate(franjas):
        desde, hasta, tasa = franja
        if base_irpf > desde:
            if i == 0:  # Primera franja no tiene monto base previo
                monto_gravable = min(base_irpf, hasta) - desde
            else:
                monto_gravable = min(base_irpf, hasta) - franjas[i-1][1]
            monto_gravable = max(monto_gravable, 0)  # Asegurar que no sea negativo
            irpf += monto_gravable * tasa

    return irpf

def calcular_carga_tributaria():
    print("=== Calculadora de Carga Tributaria ===")
    print("Ingrese la siguiente información:\n")

    # Ingresos
    print("--- Ingresos ---")
    salario_bruto = obtener_float("Sueldo bruto mensual", 25000)
    otros_ingresos = obtener_float("Otros ingresos mensuales (ej. alquiler)", 0)

    # Aguinaldo
    salario_con_aguinaldo = salario_bruto * 13 / 12

    # Salario vacacional
    tiene_salario_vacacional = obtener_bool("¿Recibe salario vacacional?")
    salario_vacacional = (salario_bruto / 30 * 20) if tiene_salario_vacacional else 0

    # Gastos básicos
    print("\n--- Gastos Básicos ---")
    gasto_ute_mensual = obtener_float("Gasto mensual en UTE (electricidad)", 0)
    tasa_ute_anual = gasto_ute_mensual * 0.03 * 12

    # Gasto en OSE (agua)
    gasto_ose_mensual = obtener_float("Gasto mensual en OSE (agua)", 0)
    tasa_saneamiento_mensual = gasto_ose_mensual * 0.125  # 12.5% de saneamiento
    tasa_saneamiento_anual = tasa_saneamiento_mensual * 12

    # Impuestos sobre propiedades
    print("\n--- Impuestos sobre Propiedades ---")
    paga_patrimonio = obtener_bool("¿Paga impuesto al patrimonio?")
    impuesto_patrimonio_anual = obtener_float("Monto anual del impuesto al patrimonio", 0) if paga_patrimonio else 0

    # Fondo de Solidaridad
    print("\n--- Fondo de Solidaridad ---")
    paga_fondo_solidaridad = obtener_bool("¿Paga Fondo de Solidaridad?")
    fondo_solidaridad_mensual = obtener_float("Monto mensual del Fondo de Solidaridad", 0) if paga_fondo_solidaridad else 0
    fondo_solidaridad_anual = fondo_solidaridad_mensual * 12

    # Caja de Profesionales
    print("\n--- Caja de Profesionales ---")
    paga_caja_profesionales = obtener_bool("¿Paga Caja de Profesionales?")
    caja_profesionales_mensual = obtener_float("Monto mensual de la Caja de Profesionales", 0) if paga_caja_profesionales else 0
    caja_profesionales_anual = caja_profesionales_mensual * 12

    # Información personal y familiar
    print("\n--- Información Personal y Familiar ---")
    tiene_hijos = obtener_bool("¿Tiene hijos?")
    cantidad_hijos = int(obtener_float("¿Cuántos hijos tiene?", 0)) if tiene_hijos else 0

    tiene_conyugue = obtener_bool("¿Tiene cónyuge o concubino a cargo?")

    # Calcular porcentaje de FONASA según situación familiar
    if tiene_conyugue:
        if tiene_hijos:
            porcentaje_fonasa = 0.08  # 8% si tiene cónyuge e hijos
        else:
            porcentaje_fonasa = 0.065  # 6.5% si tiene cónyuge pero no hijos
    else:
        if tiene_hijos:
            porcentaje_fonasa = 0.06  # 6% si no tiene cónyuge pero tiene hijos
        else:
            porcentaje_fonasa = 0.045  # 4.5% si no tiene cónyuge ni hijos

    # Información de vehículos
    print("\n--- Información de Vehículos ---")
    tiene_auto = obtener_bool("¿Tiene un auto?")
    patente = obtener_float("Costo anual de la patente", 0) if tiene_auto else 0

    # IMESI por categorías
    print("\n--- Gastos Mensuales para Cálculo de IMESI ---")
    gasto_tabaco = obtener_float("Tabaco", 0)
    gasto_alcohol = obtener_float("Bebidas alcohólicas", 0)
    gasto_bebidas_azucaradas = obtener_float("Bebidas azucaradas", 0)

    # IMESI Gasoil y Nafta (solo si tiene auto)
    gasto_gasoil = 0
    gasto_nafta = 0
    if tiene_auto:
        gasto_gasoil = obtener_float("Gasoil", 0)
        gasto_nafta = obtener_float("Nafta", 0)

    # Cálculo IMESI
    imesi_tabaco = gasto_tabaco * 0.70 * 12
    imesi_alcohol = gasto_alcohol * 0.50 * 12
    imesi_gasoil = gasto_gasoil * 0.20 * 12
    imesi_nafta = gasto_nafta * 0.30 * 12
    imesi_bebidas_azucaradas = gasto_bebidas_azucaradas * 0.20 * 12

    # Información de vivienda
    print("\n--- Información de Vivienda ---")
    tiene_inmueble = obtener_bool("¿Tiene vivienda propia?")
    alquiler = 0
    contribucion_inmobiliaria = 0
    impuesto_primaria = 0

    if tiene_inmueble:
        contribucion_inmobiliaria = obtener_float("Costo anual de la contribución inmobiliaria", 0)
        impuesto_primaria = obtener_float("Costo anual del impuesto de primaria", 0)
    else:
        alquiler = obtener_float("Costo mensual de alquiler", 10000)

    # Seguro de Accidentes Laborales
    print("\n--- Seguro de Accidentes Laborales ---")
    tiene_seguro_laboral = obtener_bool("¿Tiene seguro de accidentes laborales BPS?")
    seguro_accidentes = 0
    if tiene_seguro_laboral:
        porcentaje_seguro = obtener_float("Porcentaje del seguro de accidentes (Default: 2.01%)", 2.01)
        seguro_accidentes = salario_bruto * 12 * (porcentaje_seguro / 100)

    # Cálculos
    salario_anual_bruto = salario_bruto * 12
    salario_total_anual = salario_anual_bruto + salario_vacacional
    aportes_patronales = min(salario_bruto, 167349) * 0.075 * 12
    otros_ingresos_anual = otros_ingresos * 12
    ingreso_anual_real = salario_total_anual + aportes_patronales + otros_ingresos_anual

    # Cargas sociales
    aportes_personales = salario_total_anual * 0.15
    fonasa = salario_total_anual * porcentaje_fonasa

    # IRPF (incluye otros ingresos)
    base_irpf = salario_total_anual + otros_ingresos_anual
    irpf = calcular_irpf(base_irpf)

    # Consumo e IVA
    print("\n--- Consumo e IVA ---")
    paga_con_tarjeta_debito = obtener_bool("¿Suele pagar con tarjeta de débito (deducción del 2% de IVA)?")
    tasa_iva = 0.20 if paga_con_tarjeta_debito else 0.22  # 20% si paga con débito, 22% si no

    consumo_mensual_iva = obtener_float("Gasto mensual en consumos gravados por IVA", 0)
    consumo_anual_iva = consumo_mensual_iva * 12
    iva = consumo_anual_iva * tasa_iva

    # Total de impuestos
    imesi_total = (imesi_tabaco + imesi_alcohol + imesi_gasoil +
                   imesi_nafta + imesi_bebidas_azucaradas)

    total_impuestos = (
        aportes_patronales + aportes_personales + fonasa + irpf +
        contribucion_inmobiliaria + patente + impuesto_primaria +
        iva + imesi_total + tasa_saneamiento_anual +
        seguro_accidentes + tasa_ute_anual + impuesto_patrimonio_anual +
        fondo_solidaridad_anual + caja_profesionales_anual
    )

    porcentaje_total = total_impuestos / ingreso_anual_real * 100
    dias_trabajados_para_el_estado = round(365 * porcentaje_total / 100)

# Crear tabla de resultados
    data = {
        "Concepto": [
            "Sueldo Bruto", "Aguinaldo", "Salario Vacacional",
            "Aportes Patronales", "Ingreso Real", "Aportes Personales BPS",
            "FONASA", "IRPF", "Contribución Inmobiliaria", "Patente de Rodados",
            "Impuesto de Primaria", "Consumo gravado por IVA", "IVA sobre Consumo",
            "IMESI Tabaco", "IMESI Alcohol", "IMESI Gasoil", "IMESI Nafta",
            "IMESI Bebidas Azucaradas", "Tasa UTE (3%)", "Impuesto Patrimonio",
            "Tasa Alumbrado/Saneamiento", "Seguro Accidentes Laborales",
            "Fondo de Solidaridad", "Caja de Profesionales",
            "Total Impuestos Pagados", "Días trabajados para el Estado"
        ],
        "Valor Mensual": [
            f"${salario_bruto:,.2f}",  # Sueldo Bruto (mensual)
            f"${salario_bruto / 12:,.2f}",  # Aguinaldo dividido por 12
            f"${salario_vacacional / 12:,.2f}",  # Salario Vacacional dividido por 12
            f"${aportes_patronales / 12:,.2f}",
            f"${ingreso_anual_real / 12:,.2f}",
            f"${aportes_personales / 12:,.2f}",
            f"${fonasa / 12:,.2f}",
            f"${irpf / 12:,.2f}",
            f"${contribucion_inmobiliaria / 12:,.2f}",
            f"${patente / 12:,.2f}",
            f"${impuesto_primaria / 12:,.2f}",
            f"${consumo_mensual_iva:,.2f}",  # Consumo gravado por IVA (mensual)
            f"${iva / 12:,.2f}",
            f"${imesi_tabaco / 12:,.2f}",
            f"${imesi_alcohol / 12:,.2f}",
            f"${imesi_gasoil / 12:,.2f}" if tiene_auto else "N/A",
            f"${imesi_nafta / 12:,.2f}" if tiene_auto else "N/A",
            f"${imesi_bebidas_azucaradas / 12:,.2f}",
            f"${tasa_ute_anual / 12:,.2f}",
            f"${impuesto_patrimonio_anual / 12:,.2f}",
            f"${tasa_saneamiento_mensual / 12:,.2f}",
            f"${seguro_accidentes / 12:,.2f}",
            f"${fondo_solidaridad_mensual:,.2f}",
            f"${caja_profesionales_mensual:,.2f}",
            f"${total_impuestos / 12:,.2f}",
            f"{dias_trabajados_para_el_estado} días"
        ],
        "Valor Anual": [
            f"${salario_anual_bruto:,.2f}",  # Sueldo Bruto (anual)
            f"${salario_bruto:,.2f}",  # Aguinaldo (anual)
            f"${salario_vacacional:,.2f}",  # Salario Vacacional (anual)
            f"${aportes_patronales:,.2f}",
            f"${ingreso_anual_real:,.2f}",
            f"${aportes_personales:,.2f}",
            f"${fonasa:,.2f}",
            f"${irpf:,.2f}",
            f"${contribucion_inmobiliaria:,.2f}",
            f"${patente:,.2f}",
            f"${impuesto_primaria:,.2f}",
            f"${consumo_anual_iva:,.2f}",  # Consumo gravado por IVA (anual)
            f"${iva:,.2f}",
            f"${imesi_tabaco:,.2f}",
            f"${imesi_alcohol:,.2f}",
            f"${imesi_gasoil:,.2f}" if tiene_auto else "N/A",
            f"${imesi_nafta:,.2f}" if tiene_auto else "N/A",
            f"${imesi_bebidas_azucaradas:,.2f}",
            f"${tasa_ute_anual:,.2f}",
            f"${impuesto_patrimonio_anual:,.2f}",
            f"${tasa_saneamiento_anual:,.2f}",
            f"${seguro_accidentes:,.2f}",
            f"${fondo_solidaridad_anual:,.2f}",
            f"${caja_profesionales_anual:,.2f}",
            f"${total_impuestos:,.2f}",
            f"{dias_trabajados_para_el_estado} días"
        ]
    }

    df = pd.DataFrame(data)
    print("\nResumen de Carga Tributaria:\n")
    print(df.to_string(index=False))

    # Calcular día de liberación
    meses = {
        1: 'enero', 2: 'febrero', 3: 'marzo', 4: 'abril', 5: 'mayo', 6: 'junio',
        7: 'julio', 8: 'agosto', 9: 'septiembre', 10: 'octubre', 11: 'noviembre', 12: 'diciembre'
    }

    inicio_anio = datetime(datetime.now().year, 1, 1)
    dia_liberacion = inicio_anio + timedelta(days=dias_trabajados_para_el_estado - 1)

    print(f"\nDÍA DE LIBERACIÓN ESTATAL: {dia_liberacion.day} de {meses[dia_liberacion.month]}")

# Ejecutar la función en Google Colab
calcular_carga_tributaria()
print (" ")
print("No olvides compartir tus resulados en Twitter, y mencionarme, @PhDenLogica")

Este simulador es un trabajo en proceso,
nada de los resultados que se obtangan
tienen que ser tomados como certeros.

Cualquier duda o inquietud comunicarse por DM con

@PhDenLogica en X/Twitter

Cualquier comentario constructivo para mejorar el simulador es bienvenido